In [3]:
import imageio
import numpy as np
import matplotlib.pyplot as plt

In [6]:
def rbf_kernel(X,image_number):
    sigma_s = -0.02
    sigma_c = -0.02
    W = np.zeros((10000,10000))
    for i in range(10000):
        x1 = i//100
        y1 = i%100
        for j in range(i+1,10000):
            x2 = j//100
            y2 = j%100
            if image_number==1:
                w = np.exp(sigma_s*((x1-x2)**2+(y1-y2)**2))*np.exp(sigma_c*(np.linalg.norm(im[x1,y1]-im[x2,y2])**2))
            else:
                w = np.exp(sigma_s*((x1-x2)**2+(y1-y2)**2))*np.exp(sigma_c*(np.linalg.norm(im2[x1,y1]-im2[x2,y2])**2))
            W[i,j] = W[j,i] = w
    return W

In [ ]:
def init(method,k,image_number):
    if method == 0:
        print('Init method: random')
        res = np.zeros(10000)
        means = np.random.randint(10000,size=k)
    else:
        print('Init method: kmeans++')
        res = np.zeros(10000)
        means = np.zeros(k)
        means[0] = np.random.randint(10000)
        for i in range(1,k):
            dist = np.zeros(10000)
            for j in range(10000):
                temp = np.zeros(i)
                x = j//100
                y = j%100
                for p in range(i):
                    xm = int(means[p]//100)
                    ym = int(means[p]%100)
                    if image_number==1:
                        temp[p] = (xm-x)**2+(ym-y)**2+np.linalg.norm(im[xm,ym]-im[x,y])**2
                    else:
                        temp[p] = (xm-x)**2+(ym-y)**2+np.linalg.norm(im2[xm,ym]-im2[x,y])**2
                dist[j] = np.min(temp)
            dist = dist/dist.sum()
            means[i] = np.random.choice(10000,1,p=dist)
    
    for i in range(10000):
        temp = np.zeros(k)
        x=i//100
        y=i%100
        for j in range(k):
            xm = int(means[j]//100)
            ym = int(means[j]%100)
            if image_number==1:
                temp[j] = (xm-x)**2+(ym-y)**2+np.linalg.norm(im[xm,ym]-im[x,y])**2
            else:
                temp[j] = (xm-x)**2+(ym-y)**2+np.linalg.norm(im2[xm,ym]-im2[x,y])**2
        res[i] = np.argmin(temp)
    return res, means

In [ ]:
def classify(U,means,k,image_number):
    res = np.zeros(10000)
    for i in range(10000):
        x=i//100
        y=i%100
        temp = np.zeros(k)
        for j in range(k):
            xm = int(means[j]//100)
            ym = int(means[j]%100)
            if image_number==1:
                temp[j] = (xm-x)**2+(ym-y)**2+np.linalg.norm(im[xm,ym]-im[x,y])**2
            else:
                temp[j] = (xm-x)**2+(ym-y)**2+np.linalg.norm(im2[xm,ym]-im2[x,y])**2
        res[i] = np.argmin(temp)
    return res

In [ ]:
def draw(res,image_number,iteration,k,method):
    if method ==1:
        method = 'random'
    else:
        method = 'kmeans++'
    plt.title("Spectral, {} means after {} iters, Image {}\ninit method: {}".format(k,iteration,image_number,method))
    if image_number==1:
        plt.imshow(im.astype('int'))
    else:
        plt.imshow(im2.astype('int'))
    plt.imshow(res.reshape(100,100),alpha = 0.8)
    plt.savefig('Spectral_{}means_{}iters_image{}_{}.png'.format(k,iteration,image_number,method))

In [ ]:
def k_means(k,U,image_number):
    print("k={}".format(k))
    iteration = np.zeros(2,dtype=np.int)
    for i in range(2):
        means, res_prev = init(i,k,image_number)
        draw(res_prev,image_number,0,k,i)
        diff = prev_diff = 0
        while(1):
            iteration[i]+=1
            res = classify(U,means,k,image_number)
            draw(res,image_number,iteration[i],k,i)
            for j in range(10000):
                if res[j] != res_prev[j]:
                    diff += 1
            print('iteration: {}, diff: {}'.format(iteration[i],diff))
            if diff == prev_diff:
                break
            res_prev = res
            prev_diff = diff

In [7]:
im = imageio.imread('image1.png').astype('float32')
im2 = imageio.imread('image2.png').astype('float32')

In [ ]:
W1 = rbf_kernel(im,1)
W2 = rbf_kernel(im2,2)

In [ ]:
D1 = np.diag(np.sum(W1,axis=1))
D2 = np.diag(np.sum(W2,axis=1))
L1 = D1-W1
L2 = D2-W2
eigen_values_1, eigen_vectors_1 = np.linalg.eig(L1)
eigen_values_2, eigen_vectors_2 = np.linalg.eig(L2)

In [ ]:
U1 = 
U2 = 
k_means(k,U1,1)
k_means(k,U2,2)

In [2]:
eigen_values_1

NameError: name 'eigen_values_1' is not defined

In [ ]:
D1_inv = np.linalg.inv(D1**0.5)
D2_inv = np.linalg.inv(D2**0.5)
L1_sym = np.dot(np.dot(D1_inv,L1),D1_inv)
L2_sym = np.dot(np.dot(D2_inv,L2),D2_inv)
eigen_values_normal_1, eigen_vectors_normal_1 = np.linalg.eig(L1_sym)
eigen_values_normal_2, eigen_vectors_normal_2 = np.linalg.eig(L2_sym)

In [ ]:
U1_normal = 
U2_normal = 
k_means(k,U1_normal,1)
k_maens(k,U2_normal,2)